In [1]:
import os
import numpy as np
import toytree
import strange
import ast
from numba import jit
import toyplot

goal: take a sequence of newick trees (organized in a **strange** directory) and, for each tree, determine if a specific bipartition is present

Then, examine the fragment lengths for which each split is preserved.

First, make a tree as usual.

In [2]:
tree = toytree.rtree.unittree(ntips=13, treeheight=3, seed=42)
tree.draw();

<svg class="toyplot-canvas-Canvas" height="275.0px" id="t19e3c7c805994893acd82b0df2f4eee6" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 265.0 275.0" width="265.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> r3 r12 r8 r9 r0 r11 r5 r6 r4 r7 r10 r1 r2

Define a coalseq object and make a treeseq.

In [3]:
sim = strange.Coalseq(tree,
            'testdir',
           recombination_rate = 1e-9,
           length = 1000000)
sim.make_treeseq()

Directory 'testdir' created.


In [4]:
sim.treeseq.num_trees

9539

Write out all of these newick genetrees.

In [5]:
sim.write_trees()

Directory 'testdir/ms_genetrees' created.


For the purposes of this, we actually don't need to write out the sequences. That saves us some time.

In [6]:
#sim.write_seqs()

In [7]:
#sim.build_seqs(hdf5=True)

Instead, we will now use "write_clades()" which will make a new "clades" directory with as many files as we have gene trees. In each file will be a list of clades for that particular gene tree.

In [8]:
sim.write_clades()

Directory 'testdir/clades' created.


Now we're going to ask about the persistence of each clade, in an exploratory way. So first: how many clades are there among all the gene trees?

In [85]:
full_list = []
for i in os.listdir('testdir/clades/'):
    with open('testdir/clades/'+i,'r') as f:
        data=f.read().splitlines()
    test = [sorted(ast.literal_eval(i)) for i in data]
    full_list.extend(test)
    full_list = list(np.unique(full_list))

Show some of them

In [75]:
full_list[0:10]

[['1', '10'],
 ['1', '10', '11'],
 ['1', '10', '11', '12', '13'],
 ['1', '10', '11', '12', '13', '2'],
 ['1', '10', '11', '12', '13', '2', '3'],
 ['1', '10', '11', '12', '13', '2', '3', '4'],
 ['1', '10', '11', '12', '13', '2', '3', '4', '5'],
 ['1', '10', '11', '12', '13', '2', '3', '4', '5', '6'],
 ['1', '10', '11', '12', '13', '2', '3', '4', '5', '6', '7'],
 ['1', '10', '11', '12', '13', '2', '3', '4', '5', '6', '7', '8']]

How many total?

In [76]:
len(full_list)

1481

Wow -- a lot.

Let's sort the list by clade size -- just for fun.

In [122]:
full_list=np.array(full_list)[np.argsort([len(i) for i in full_list])]

In [123]:
full_list

array([list(['1', '10']), list(['3', '8']), list(['3', '9']), ...,
       list(['1', '10', '11', '12', '13', '2', '3', '4', '5', '7', '8', '9']),
       list(['1', '10', '11', '12', '13', '2', '3', '4', '5', '6', '8', '9']),
       list(['1', '10', '12', '13', '2', '3', '4', '5', '6', '7', '8', '9'])],
      dtype=object)

Now, we'd love to go through and say which of these clades (by index) are present in which gene trees.

In [126]:
@jit
def get_equal_list(big_list,little_list):
    for i in range(len(big_list)):
        if big_list[i] == little_list:
            return(i)

Now make a huge array to contain all of the presence vs. absence of particular clades...

In [131]:
clade_pres = np.zeros((len(full_list),len(os.listdir('testdir/clades/'))),dtype=np.int8)

In [132]:
clade_pres.shape

(1481, 9539)

So here, each row corresponds to a clade. Each column is a consecutive gene tree. And since we sorted the clades by length, the shorter clades (e.g. of two taxa) will be near the top, and the longer clades (e.g. of 12 taxa) will be near the bottom of the array.

We'll change a cell from 0 to 1 if that clade (row) is present in that gene tree (col).

In [139]:
for i in range(len(os.listdir('testdir/clades/'))):
    with open('testdir/clades/'+str(i)+'.txt','r') as f:
        data=f.read().splitlines()
    test = [sorted(ast.literal_eval(dat)) for dat in data]
    for clade in test:
        row = get_equal_list(full_list,clade)
        clade_pres[row,i] = 1

Now we have a huge numpy array where each column corresponds to a particular gene tree, and each row corresponds to a possible clade. Cells with "1" in this array indicate that the particular clade is found in the particular gene tree.

We can visualize this using toyplot matrix:

In [169]:
toyplot.matrix(clade_pres[10:20,0:1000],width = 1300, height = 300);

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 277 278 279 280 281 282 283 284 285 286 287 288 289 290 291 292 293 294 295 296 297 298 299 300 301 302 303 304 305 306 307 308 309 310 311 312 313 314 315 316 317 318 319 320 321 322 323 324 325 326 327 328 329 330 331 332 333 334 335 336 337 338 339 340 341 342 343 344 345 346 347 348 349 350 351 352 353 354 355 356 357 358 359 360 361 362 363 364 365 366 367 368 369 370 371 372 373 374 375 376 377 378 379 380 381 382 383 384 385 386 387 388 389 390 391 392 393 394 395 396 397 398 399 400 401 402 403 404 405 406 407 408 409 410 411 412 413 414 415 416 417 418 419 420 421 422 423 424 425 426 427 428 429 430 431 432 433 434 435 436 437 438 439 440 441 442 443 444 445 446 447 448 449 450 451 452 453 454 455 456 457 458 459 460 461 462 463 464 465 466 467 468 469 470 471 472 473 474 475 476 477 478 479 480 481 482 483 484 485 486 487 488 489 490 491 492 493 494 495 496 497 498 499 500 501 502 503 504 505 506 507 508 509 510 511 512 513 514 515 516 517 518 519 520 521 522 523 524 525 526 527 528 529 530 531 532 533 534 535 536 537 538 539 540 541 542 543 544 545 546 547 548 549 550 551 552 553 554 555 556 557 558 559 560 561 562 563 564 565 566 567 568 569 570 571 572 573 574 575 576 577 578 579 580 581 582 583 584 585 586 587 588 589 590 591 592 593 594 595 596 597 598 599 600 601 602 603 604 605 606 607 608 609 610 611 612 613 614 615 616 617 618 619 620 621 622 623 624 625 626 627 628 629 630 631 632 633 634 635 636 637 638 639 640 641 642 643 644 645 646 647 648 649 650 651 652 653 654 655 656 657 658 659 660 661 662 663 664 665 666 667 668 669 670 671 672 673 674 675 676 677 678 679 680 681 682 683 684 685 686 687 688 689 690 691 692 693 694 695 696 697 698 699 700 701 702 703 704 705 706 707 708 709 710 711 712 713 714 715 716 717 718 719 720 721 722 723 724 725 726 727 728 729 730 731 732 733 734 735 736 737 738 739 740 741 742 743 744 745 746 747 748 749 750 751 752 753 754 755 756 757 758 759 760 761 762 763 764 765 766 767 768 769 770 771 772 773 774 775 776 777 778 779 <text style="fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:helvetica;font-size:12.0px;font-weight:normal;stroke:none;vertical-align:baseline;white-space

So you can see that some particular clades tend to be present in a lot of consecutive gene trees (they aren't split up too often)